# Lab: Catalogando PostgreSQL no Apache Atlas

## A Importância do Catálogo de Dados

### Por que Catalogar Dados?

Em organizações modernas, os dados estão espalhados por múltiplos sistemas, formatos e localizações. Um **catálogo de dados** é essencial para:

- **Descoberta**: Encontrar rapidamente datasets relevantes
- **Compreensão**: Entender estrutura, significado e qualidade dos dados
- **Linhagem**: Rastrear origem e transformações dos dados
- **Governança**: Aplicar políticas de segurança e compliance
- **Colaboração**: Facilitar compartilhamento entre equipes
- **Produtividade**: Reduzir tempo de análise e desenvolvimento

### Metadados: O DNA dos Dados

Os **metadados** são informações sobre os dados que incluem:

- **Estruturais**: Esquemas, tipos, relacionamentos
- **Descritivos**: Nomes, descrições, tags, glossários
- **Operacionais**: Frequência de uso, performance, qualidade
- **Administrativos**: Proprietários, permissões, políticas

## Apache Atlas: História e Evolução

### Origens (2015-2017)

O **Apache Atlas** nasceu da necessidade da Hortonworks de gerenciar metadados no ecossistema Hadoop:

- **2015**: Projeto iniciado pela Hortonworks
- **2016**: Doação para Apache Software Foundation
- **2017**: Primeira versão estável (0.8)

### Características Principais

- **Governança Unificada**: Catálogo centralizado para Big Data
- **Linhagem Automática**: Rastreamento de transformações
- **Classificação**: Tags e tipos personalizados
- **Busca Avançada**: Interface web intuitiva
- **Integração**: Conectores para Hive, HBase, Kafka, etc.

### Arquitetura

- **Core**: Motor de metadados baseado em grafos
- **Storage**: HBase para persistência
- **Search**: Apache Solr para indexação
- **Messaging**: Apache Kafka para eventos
- **Security**: Integração com Ranger e Kerberos

### Casos de Uso

- **Data Lakes**: Catalogação de datasets em HDFS
- **Data Warehouses**: Metadados de tabelas e views
- **ETL Pipelines**: Linhagem de transformações
- **Compliance**: Auditoria e classificação de dados sensíveis

## Objetivo do Lab

Neste laboratório, vamos:

1. **Conectar** ao banco PostgreSQL Northwind
2. **Extrair** metadados de tabelas e colunas
3. **Catalogar** no Apache Atlas via API REST
4. **Visualizar** o catálogo na interface web

**Resultado**: Catálogo completo com 14 tabelas e suas colunas integradas ao Atlas.


## Passo 1: Configuração e Importações

Esta célula configura todas as dependências e parâmetros necessários para conectar ao PostgreSQL e Apache Atlas.

**O que faz:**
- Importa bibliotecas para HTTP, banco de dados e manipulação de dados
- Define URLs e credenciais de conexão
- Configura autenticação HTTP Basic para Atlas

In [ ]:
# Importação das bibliotecas necessárias
import requests                    # Para fazer requisições HTTP ao Atlas
from requests.auth import HTTPBasicAuth  # Para autenticação HTTP Basic
import psycopg2                   # Driver para conectar ao PostgreSQL
import pandas as pd               # Para manipulação de dados em DataFrames
import json                       # Para trabalhar com dados JSON

# Configurações do Apache Atlas
ATLAS_URL = "http://atlas:21000"  # URL do servidor Atlas (container Docker)
ATLAS_USER = "admin"              # Usuário padrão do Atlas
ATLAS_PASSWORD = "admin"          # Senha padrão do Atlas

# Configurações do PostgreSQL
POSTGRES_CONFIG = {
    "host": "postgres_erp",        # Nome do container PostgreSQL
    "port": 5432,                  # Porta padrão do PostgreSQL
    "database": "northwind",       # Nome do banco de dados
    "user": "postgres",            # Usuário do banco
    "password": "postgres"         # Senha do banco
}

# Configurar autenticação HTTP Basic para Atlas
auth = HTTPBasicAuth(ATLAS_USER, ATLAS_PASSWORD)
print("✅ Configurado")

## Passo 2: Extração de Metadados do PostgreSQL

Esta célula conecta ao PostgreSQL e extrai metadados estruturais de todas as tabelas.

**O que faz:**
- Conecta ao banco PostgreSQL Northwind
- Consulta o `information_schema` para obter lista de tabelas
- Para cada tabela, extrai informações das colunas (nome, tipo, nullable)
- Organiza os metadados em um dicionário Python

In [ ]:
def get_postgres_metadata():
    """Extrai metadados estruturais do PostgreSQL"""
    # Estabelece conexão com o PostgreSQL usando as configurações definidas
    conn = psycopg2.connect(**POSTGRES_CONFIG)
    
    # Query SQL para buscar todas as tabelas do schema 'public'
    tables_df = pd.read_sql("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'  -- Apenas tabelas do schema público
        ORDER BY table_name            -- Ordena alfabeticamente
    """, conn)
    
    # Dicionário para armazenar metadados de todas as tabelas
    metadata = {}
    
    # Itera sobre cada tabela encontrada
    for _, row in tables_df.iterrows():
        table_name = row['table_name']  # Nome da tabela atual
        
        # Query SQL para buscar colunas da tabela específica
        columns_df = pd.read_sql("""
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns 
            WHERE table_name = %s AND table_schema = 'public'
            ORDER BY ordinal_position  -- Ordena pela posição original da coluna
        """, conn, params=[table_name])  # Usa parâmetro para evitar SQL injection
        
        # Converte DataFrame para lista de dicionários e armazena
        metadata[table_name] = columns_df.to_dict('records')
    
    # Fecha a conexão com o banco
    conn.close()
    return metadata

# Executa a extração de metadados
postgres_metadata = get_postgres_metadata()
print(f"📋 {len(postgres_metadata)} tabelas encontradas")
print(f"Exemplo: {list(postgres_metadata.keys())[:3]}")

## 🗄️ Passo 3: Criação do Database no Atlas

Esta célula cria ou localiza o database no Apache Atlas que representará nosso PostgreSQL.

**O que faz:**
- Tenta criar um novo database no Atlas usando a API REST
- Se já existir, busca o GUID do database existente
- Retorna o identificador único (GUID) do database para usar nas próximas etapas

In [ ]:
def create_database_in_atlas():
    """Cria ou localiza database no Atlas"""
    # Payload JSON para criar entidade do tipo 'hive_db' no Atlas
    db_payload = {
        "entities": [{
            "typeName": "hive_db",                    # Tipo de entidade (database)
            "attributes": {
                "name": "northwind_postgres",          # Nome do database
                "qualifiedName": "northwind_postgres@cluster1",  # Nome único global
                "clusterName": "cluster1"              # Nome do cluster
            },
            "guid": -1                                 # GUID temporário (será gerado pelo Atlas)
        }]
    }
    
    # Tenta criar o database via API REST do Atlas
    response = requests.post(f"{ATLAS_URL}/api/atlas/v2/entity/bulk", 
                            json=db_payload, auth=auth)
    
    # Se criação foi bem-sucedida, retorna o GUID gerado
    if response.status_code in [200, 201]:
        result = response.json()
        if 'mutatedEntities' in result and 'CREATE' in result['mutatedEntities']:
            return result['mutatedEntities']['CREATE'][0]['guid']
    
    # Se não conseguiu criar, busca database existente
    search_response = requests.get(f"{ATLAS_URL}/api/atlas/v2/search/basic", 
                                 params={"query": "northwind_postgres"}, auth=auth)
    
    # Processa resultado da busca
    if search_response.status_code == 200:
        entities = search_response.json().get('entities', [])
        # Filtra apenas entidades do tipo 'hive_db'
        db_entities = [e for e in entities if e.get('typeName') == 'hive_db']
        if db_entities:
            return db_entities[0]['guid']  # Retorna GUID do primeiro database encontrado
    
    return None  # Retorna None se não conseguiu criar nem encontrar

# Executa criação/busca do database
db_guid = create_database_in_atlas()
print(f"✅ Database GUID: {db_guid}")

## Passo 4: Catalogação Completa de Tabelas e Colunas

Esta célula implementa a catalogação integrada de tabelas com suas colunas no Atlas.

**O que faz:**
- Remove tabelas existentes para evitar duplicatas
- Cria tabelas e colunas em uma única operação
- Usa GUIDs negativos para referenciar tabelas nas colunas
- Garante que as colunas apareçam no schema das tabelas no Atlas

In [ ]:
def catalog_data(table_name, columns, db_guid):
    """Recria tabela com colunas integradas no Atlas"""
    
    # Primeiro, remove tabela existente se houver
    search_response = requests.get(f"{ATLAS_URL}/api/atlas/v2/search/basic", 
                                  params={"query": table_name}, auth=auth)
    
    # Se encontrou entidades, deleta as tabelas existentes
    if search_response.status_code == 200:
        entities = search_response.json().get('entities', [])
        for entity in entities:
            # Verifica se é uma tabela com o nome exato
            if entity.get('typeName') == 'hive_table' and entity.get('displayText') == table_name:
                # Deleta a entidade usando seu GUID
                requests.delete(f"{ATLAS_URL}/api/atlas/v2/entity/guid/{entity['guid']}", auth=auth)
                print(f"  🗑️ Deletada tabela existente: {table_name}")
    
    # Lista para armazenar todas as entidades (tabela + colunas)
    entities = []
    
    # Cria entidade da tabela
    table_entity = {
        "typeName": "hive_table",                    # Tipo de entidade (tabela)
        "attributes": {
            "name": table_name,                       # Nome da tabela
            "qualifiedName": f"northwind_postgres.{table_name}@cluster1",  # Nome único
            "db": {"guid": db_guid},                  # Referência ao database pai
            "owner": "postgres"                       # Proprietário da tabela
        },
        "guid": -1                                    # GUID temporário negativo
    }
    entities.append(table_entity)
    
    # Cria entidades das colunas
    for i, col in enumerate(columns, 1):              # Enumera começando do 1
        column_entity = {
            "typeName": "hive_column",                # Tipo de entidade (coluna)
            "attributes": {
                "name": col['column_name'],            # Nome da coluna
                "qualifiedName": f"northwind_postgres.{table_name}.{col['column_name']}@cluster1",
                "table": {"guid": -1},                 # Referência à tabela (GUID -1)
                "type": col['data_type'],              # Tipo de dados da coluna
                "position": i                          # Posição da coluna na tabela
            },
            "guid": -(i+1)                             # GUID temporário negativo único
        }
        entities.append(column_entity)
    
    # Envia todas as entidades (tabela + colunas) em uma única requisição
    payload = {"entities": entities}
    response = requests.post(f"{ATLAS_URL}/api/atlas/v2/entity/bulk", 
                           json=payload, auth=auth)
    
    # Verifica se a criação foi bem-sucedida
    if response.status_code in [200, 201]:
        result = response.json()
        created = result.get('mutatedEntities', {}).get('CREATE', [])
        print(f"  ✅ {table_name}: {len(created)} entidades criadas")
        return True
    else:
        print(f"  ❌ Erro {table_name}: {response.status_code}")
        return False

# Executa catalogação de todas as tabelas
if db_guid:
    print(f"🔄 Recriando {len(postgres_metadata)} tabelas com colunas integradas...")
    success_count = 0
    
    # Processa cada tabela individualmente
    for table_name, columns in postgres_metadata.items():
        print(f"\n📋 Processando: {table_name}")
        success = catalog_data(table_name, columns, db_guid)
        if success:
            success_count += 1
    
    print(f"\n✅ {success_count}/{len(postgres_metadata)} tabelas recriadas com sucesso!")
    print("   Verifique no Atlas: todas as colunas devem aparecer no schema das tabelas")
else:
    print("❌ Database não disponível")

## Passo 5: Verificação Final do Catálogo

Esta célula verifica o resultado final da catalogação no Apache Atlas.

**O que faz:**
- Busca todas as entidades relacionadas ao nosso projeto
- Conta databases, tabelas e colunas catalogadas
- Exibe estatísticas finais e link para acesso ao Atlas
- Confirma que a catalogação foi bem-sucedida

In [ ]:
# Busca todas as entidades relacionadas ao projeto northwind_postgres
search_response = requests.get(f"{ATLAS_URL}/api/atlas/v2/search/basic", 
                              params={"query": "*", "limit": 200}, auth=auth)

# Processa e exibe resultados da catalogação
if search_response.status_code == 200:
    entities = search_response.json().get('entities', [])
    
    # Filtra entidades por tipo e status ativo
    databases = [e for e in entities if e.get('typeName') == 'hive_db' and e.get('status') == 'ACTIVE']
    tables = [e for e in entities if e.get('typeName') == 'hive_table' and e.get('status') == 'ACTIVE']
    columns = [e for e in entities if e.get('typeName') == 'hive_column' and e.get('status') == 'ACTIVE']
    
    # Exibe estatísticas finais
    print("📊 Resultado Final do Catálogo:")
    print(f"  🗄️ Databases: {len(databases)}")
    print(f"  📋 Tabelas: {len(tables)}")
    print(f"  📝 Colunas: {len(columns)}")
    
    # Mostra algumas tabelas como exemplo
    if tables:
        print("\n📋 Tabelas catalogadas (primeiras 5):")
        for i, table in enumerate(tables[:5], 1):
            print(f"  {i}. {table.get('displayText')}")
    
    # Informações de acesso ao Atlas
    print(f"\n🎉 Acesse o Atlas: http://localhost:21000")
    print(f"   Usuário: admin | Senha: admin")
    print(f"\n💡 Dica: Navegue até 'Search' e busque por 'northwind_postgres' para ver o catálogo completo")
else:
    print(f"❌ Erro na verificação: {search_response.status_code}")

## Conclusão

### O que Conseguimos

- **Extração Automática**: Metadados do PostgreSQL via `information_schema`  
- **Catalogação Completa**: 14 tabelas + colunas no Apache Atlas  
- **Integração Perfeita**: Colunas visíveis no schema das tabelas  
- **API REST**: Automação via APIs do Atlas  

### Próximos Passos

- **Classificações**: Adicionar tags e classificações aos dados
- **Linhagem**: Mapear transformações e fluxos de dados
- **Glossário**: Criar definições de negócio para termos
- **Automação**: Integrar com pipelines de CI/CD

### Benefícios Alcançados

- **Descoberta**: Dados facilmente encontráveis  
- **Documentação**: Metadados estruturados e acessíveis  
- **Governança**: Base para políticas de dados  
- **Produtividade**: Redução do tempo de análise  

**Parabéns! Você criou seu primeiro catálogo de dados com Apache Atlas!**